### **Cuda Programming Applications**

This mini-lab targets some hands-on implementations and more practice on cuda in common real-world recurring tasks. Moreover, we aim to compare the outcomes of our low-level implementations with the built-in functions in popular frameworks as Pytorch.

### **Requirement**

A) A cuda program is required to carry out a 3D convolution over RGB images and save the output ones, the program is given a path to a folder containing the input images and that of an output folder that should contain the outputs, respectively as command line arguments.

1.   kernel1: basic implementation (no tiling)
2.   kernel2: tiling where each block matches the input tile size.

Notes:
*   Add necessary paddings so that the output image size is the same as that of the input one.

*   The kernel should be able to handle a batch of images at a time, the batch size is passed as the 3rd argument.
*   The mask is given in a .txt file, whose path is passed as the 4th argument. The first line contains its dimension n (one number only as it's a square mask) then the consecutive n lines contain the mask rows, each row in a separate line.

  Ex: ./a.out input_folder_path output_folder_path 4 mask.txt

B) Implement the same program in python, using the built-in convolution functions in Pytorch.

C) Profile each program carefully and do sufficient experiments to compare between them and collect insightful results. Organise your results in a tabular form and prepare a comprehensive report with visual graphs explaining all of your findings. Also mention the impact of declaring the mask as constant in terms of execution time and elaborate on this in your report.

#### **Helpers**

This section contains some helpers that could be needed for the requirement. Check it frequently.

**Helper1**: Read RGB images in C

In [1]:

!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0vip98iw
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0vip98iw
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-2ua8qwry/wheels/ef/1d/c6/f7e47f1aa1bc9d05c4120d94f90a79cf28603ef343b0dd43ff
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp8u61tzi_".


In [2]:
# Fetch stb_image library
!git clone https://github.com/nothings/stb.git
!cp stb/stb_image.h /usr/local/include/
!cp stb/stb_image_write.h /usr/local/include/

Cloning into 'stb'...
remote: Enumerating objects: 8138, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8138 (delta 1), reused 0 (delta 0), pack-reused 8132 (from 1)
Receiving objects: 100% (8138/8138), 5.64 MiB | 9.98 MiB/s, done.
Resolving deltas: 100% (5400/5400), done.


# **kernel1**

In [ ]:
%%writefile kernel1.cu
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <fstream>
#include <assert.h>
#include <string.h>
#include <dirent.h>
#include <vector>
#include <string>
#include <float.h>
#include<iostream>
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#include <sys/stat.h>
__constant__ float c_mask[256];


void save_images(const char* output_folder, float* output_data,   int width, int height, int channels,  int batch_size,std::vector<std::string> input_paths,int batch_start) {
    // Create output directory if it doesn't exist
    mkdir(output_folder, 0777);


    // Process each image in the current batch
    for (int i = 0; i < batch_size; i++) {

        // Extract filename from input path
        std::string path = input_paths[ i+batch_start];

        size_t last_slash = path.find_last_of("/\\");
        std::string filename = (last_slash == std::string::npos) ? path : path.substr(last_slash + 1);

        // Create output path (preserve extension)
        std::string output_path = std::string(output_folder) + "/conv_" + filename;

        // Allocate memory for output image (convert from float to uint8)
        unsigned char* image_data = (unsigned char*)malloc(width * height*channels );

      float min_pixel = FLT_MAX;
      float max_pixel = -FLT_MAX;

  for (int k=0;k<channels;k++)
       for (int y = 0; y < height; y++) {
            for (int x = 0; x < width; x++) {

          int output_idx = ((i*channels+k )* height * width ) +        (y * width ) +   (x ) ;
            if (output_data[output_idx] < min_pixel)
                min_pixel = output_data[output_idx];
            if (output_data[output_idx] > max_pixel)
                max_pixel = output_data[output_idx];
        }}
        // Convert and normalize output data
        for(int k=0;k<channels;k++)
        for (int y = 0; y < height; y++) {
            for (int x = 0; x < width; x++) {
              {
                    // Calculate indices (NHWC layout)
                    int output_idx = ((i*channels+k ) * height * width ) +
                                   (y * width ) +
                                   (x ) ;

                    float pixel_val = output_data[output_idx];

                    pixel_val=static_cast<unsigned char>(255.0f *(pixel_val-min_pixel)/(max_pixel-min_pixel));
                   image_data[((y * width + x)*channels+k) ] = pixel_val;
                }
            }
        }

        // Save image (preserve original format)
        std::string ext = filename.substr(filename.find_last_of(".") + 1);
        int success = 0;
        if (ext == "png") {
            success = stbi_write_png(output_path.c_str(), width, height, channels, image_data, width * channels);
        }
        else if (ext == "jpg" || ext == "jpeg") {
            success = stbi_write_jpg(output_path.c_str(), width, height, channels, image_data, 90);  // 90% quality
        }
        else {
            printf("Unsupported output format for %s, defaulting to PNG\n", output_path.c_str());
            success = stbi_write_png(output_path.c_str(), width, height, 1, image_data, width * 1);
        }

        if (!success) {
            printf("Failed to save image %s\n", output_path.c_str());
        }

        free(image_data);
    }
}

__global__ void conv3D_basic(const uint8_t *input, int width, int height, int depth,int batch_size, float *output, float *mask, int maskWidth)
 {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int batch_index= threadIdx.z+blockIdx.z * blockDim.z;


    if (col >= width || row >= height||batch_index>=batch_size ) return;


   for (int channel=0;channel<depth;channel++)
{
   float sum = 0.0f;
    for (int i = 0; i < maskWidth; ++i) {
        for (int j =0; j < maskWidth; ++j) {

            int curr_row = row+i-maskWidth/2;
            int curr_col = col+j-maskWidth/2;
            if(curr_col<width&& curr_row<height&&curr_col>=0&&curr_row>=0)
            {

             {
               sum+=mask[i*maskWidth+j]*static_cast<float>(input[batch_index*height*width*depth  +  curr_row*width*depth+curr_col*depth+channel]);

}
            }
        }
    }
      int outIdx = (batch_index*depth+channel)*height*width+row*width+col;
               output[outIdx] = sum;}

}


float* read_mask(const char* file_path, int& maskWidth) {
    FILE* file = fopen(file_path, "r");
    if (!file) {
        fprintf(stderr, "Error: Could not open mask file %s\n", file_path);
        return nullptr;
    }

    // Read mask dimension (first line)
    if (fscanf(file, "%d", &maskWidth) != 1) {
        fprintf(stderr, "Error: Could not read mask dimension from %s\n", file_path);
        fclose(file);
        return nullptr;
    }

    float* mask = (float*)malloc(maskWidth * maskWidth * sizeof(float));
    if (!mask) {
        fprintf(stderr, "Error: Memory allocation failed for mask\n");
        fclose(file);
        return nullptr;
    }

    // Read mask values (subsequent lines)
    for (int i = 0; i < maskWidth; i++) {
        for (int j = 0; j < maskWidth; j++) {
            if (fscanf(file, "%f", &mask[i * maskWidth + j]) != 1) {
                fprintf(stderr, "Error: Invalid mask data at row %d, column %d\n", i+1, j+1);
                free(mask);
                fclose(file);
                return nullptr;
            }
        }
    }

    fclose(file);
    return mask;
}


uint8_t* load_images(const char* folder_path, int& width, int& height, int& channels, int batch_size,int & num_images, std::vector<std::string>& image_paths) {
    DIR *dir;
    struct dirent *ent;

    if ((dir = opendir(folder_path)) != NULL) {
        while ((ent = readdir(dir)) != NULL) {
            std::string filename = ent->d_name;
            if (filename.find(".jpg") != std::string::npos ||
                filename.find(".jpeg") != std::string::npos ||
                filename.find(".png") != std::string::npos) {
                image_paths.push_back(std::string(folder_path) + "/" + filename);
            }
        }
        closedir(dir);
    } else {
        perror("Could not open directory");
        return nullptr;
    }

    if (image_paths.empty()) {
        printf("No images found in %s\n", folder_path);
        return nullptr;
    }

   num_images=image_paths.size();
   uint8_t* h_input;
    // Load images into batch
    for (int i = 0; i < image_paths.size(); i++) {
        int img_width, img_height, img_channels;
        unsigned char* image_data = stbi_load(image_paths[i].c_str(), &img_width, &img_height, &img_channels, 0);

        if(i==0)
        {
           height=img_height;
        width=img_width;
        channels=img_channels;
           size_t input_size = image_paths.size() * height * width * channels * sizeof(uint8_t);
            h_input = (uint8_t*)malloc(input_size);

        }
        if (!image_data) {
            printf("Failed to load image: %s\n", image_paths[i].c_str());
            continue;
        }


        // Copy image data to batch (NHWC layout)
        for (int y = 0; y < height; y++) {
            for (int x = 0; x < width; x++) {
                for (int c = 0; c < channels; c++) {
                    int src_idx = (y * width + x) * channels + c;
                    int dst_idx = (i * height * width * channels) +
                                 (y * width * channels) +
                                 (x * channels) + c;
                    h_input[dst_idx] = image_data[src_idx];
                }
            }
        }

        stbi_image_free(image_data);
    }

    return h_input;
}

int main(int argc, char** argv)
{

   if (argc != 5) {
        printf("arguments are incorrect");
        return 1;
    }
    const char* input_folder = argv[1];
    const char* output_folder = argv[2];
    int batch_size = atoi(argv[3]);
    const char* mask_file = argv[4];



    int maskWidth;
    float*h_mask=read_mask(mask_file,maskWidth);
    if(!h_mask)
    {
      return 1;

    }


    int height,width,depth;
    uint8_t* h_input;
    int num_images;
    std::vector<std::string> input_paths;
    h_input=load_images(input_folder,width,height,depth,batch_size, num_images,input_paths);






    uint8_t* d_input;
    float* d_output;
    float* d_mask;

    //Allocate


    cudaMalloc(&d_mask, maskWidth * maskWidth * sizeof(float));
    cudaMemcpy(d_mask, h_mask, maskWidth * maskWidth * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(c_mask, h_mask, maskWidth * maskWidth * sizeof(float));

    for(int batch_start=0;batch_start<num_images;batch_start+=batch_size)
    {

        int current_batch_size = (batch_start + batch_size > num_images) ? num_images - batch_start : batch_size;
        size_t input_size = current_batch_size * height * width * sizeof(uint8_t)*depth;
        size_t output_size = current_batch_size * height * width * sizeof(float)*depth;;
         float* h_output = (float*)malloc(output_size);

            //copy to gpu
            cudaMalloc(&d_input, input_size);
            cudaMalloc(&d_output, output_size);


            cudaMemcpy(d_input,  &h_input[batch_start * width * height * depth], input_size, cudaMemcpyHostToDevice);

   dim3 block_size(16, 16, 1);
   dim3 grid_size(
       (width + block_size.x - 1) / block_size.x,
       (height + block_size.y - 1) / block_size.y,
       current_batch_size

   );


            conv3D_basic<<<grid_size, block_size>>>(d_input, width, height, depth, current_batch_size,
                d_output, d_mask, maskWidth);

            cudaError_t err = cudaGetLastError();
            if (err != cudaSuccess) {
            printf("CUDA error: %s\n", cudaGetErrorString(err));
            }

            cudaDeviceSynchronize();  // Required to flush printf output


            cudaMemcpy(h_output, d_output, output_size, cudaMemcpyDeviceToHost);


            save_images(output_folder,h_output,width,height,depth,current_batch_size,input_paths,batch_start);
            cudaFree(d_input);
            cudaFree(d_output);
            free(h_output);


    }



  free(h_mask);
    free(h_input);
    cudaFree(d_mask);

    return 0;


}


Overwriting kernel1.cu


In [101]:
!nvcc -arch=sm_75 kernel1.cu -o kernel1.out
!nvprof ./kernel1.out /content/input /content/kernel11 4 /content/mask.txt


==26149== NVPROF is profiling process 26149, command: ./kernel1.out /content/input /content/kernel11 4 /content/mask.txt
==26149== Profiling application: ./kernel1.out /content/input /content/kernel11 4 /content/mask.txt
==26149== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   90.46%  83.627ms         2  41.814ms  26.617ms  57.010ms  [CUDA memcpy DtoH]
                    6.52%  6.0255ms         4  1.5064ms     640ns  4.0796ms  [CUDA memcpy HtoD]
                    3.02%  2.7942ms         2  1.3971ms  934.10us  1.8601ms  conv3D_basic(unsigned char const *, int, int, int, int, float*, float*, int)
      API calls:   64.67%  178.97ms         5  35.794ms  101.36us  178.42ms  cudaMalloc
                   33.35%  92.286ms         5  18.457ms  21.388us  58.009ms  cudaMemcpy
                    1.02%  2.8274ms         2  1.4137ms  965.86us  1.8615ms  cudaDeviceSynchronize
                    0.77%  2.1443ms         5  4

# **kernel2**

In [105]:
%%writefile kernel2.cu
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <fstream>
#include <assert.h>
#include <string.h>
#include <dirent.h>
#include <vector>
#include <string>
#include <float.h>
#include<iostream>
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#include <sys/stat.h>
# define OUTPUT_TILE_DIM 16
__constant__ float c_mask[256];
void save_images(const char* output_folder, float* output_data,   int width, int height, int channels,  int batch_size,std::vector<std::string> input_paths,int batch_start) {
    // Create output directory if it doesn't exist
    mkdir(output_folder, 0777);


    // Process each image in the current batch
    for (int i = 0; i < batch_size; i++) {

        // Extract filename from input path
        std::string path = input_paths[ i+batch_start];

        size_t last_slash = path.find_last_of("/\\");
        std::string filename = (last_slash == std::string::npos) ? path : path.substr(last_slash + 1);

        // Create output path (preserve extension)
        std::string output_path = std::string(output_folder) + "/conv_" + filename;

        // Allocate memory for output image (convert from float to uint8)
        unsigned char* image_data = (unsigned char*)malloc(width * height*channels );

      float min_pixel = FLT_MAX;
       float max_pixel = -FLT_MAX;

  for (int k=0;k<channels;k++)
       for (int y = 0; y < height; y++) {
            for (int x = 0; x < width; x++) {

          int output_idx = ((i*channels+k )* height * width ) +        (y * width ) +   (x ) ;
            if (output_data[output_idx] < min_pixel)
                min_pixel = output_data[output_idx];
            if (output_data[output_idx] > max_pixel)
                max_pixel = output_data[output_idx];
        }}
        // Convert and normalize output data
        for(int k=0;k<channels;k++)
        for (int y = 0; y < height; y++) {
            for (int x = 0; x < width; x++) {
              {
                    // Calculate indices (NHWC layout)
                    int output_idx = ((i*channels+k ) * height * width ) +
                                   (y * width ) +
                                   (x ) ;

                    float pixel_val = output_data[output_idx];

                    pixel_val=static_cast<unsigned char>(255.0f *(pixel_val-min_pixel)/(max_pixel-min_pixel));
                   image_data[((y * width + x)*channels+k) ] = pixel_val;
                }
            }
        }

        // Save image (preserve original format)
        std::string ext = filename.substr(filename.find_last_of(".") + 1);
        int success = 0;
        if (ext == "png") {
            success = stbi_write_png(output_path.c_str(), width, height, channels, image_data, width * channels);
        }
        else if (ext == "jpg" || ext == "jpeg") {
            success = stbi_write_jpg(output_path.c_str(), width, height, channels, image_data, 90);  // 90% quality
        }
        else {
            printf("Unsupported output format for %s, defaulting to PNG\n", output_path.c_str());
            success = stbi_write_png(output_path.c_str(), width, height, 1, image_data, width * 1);
        }

        if (!success) {
            printf("Failed to save image %s\n", output_path.c_str());
        }

        free(image_data);
    }
}

__global__ void conv3D_tiled(const uint8_t *input, int width, int height, int depth,int batch_size, float *output, float *mask, int maskWidth)
 {
    int  input_tile_dim=OUTPUT_TILE_DIM+maskWidth-1;
     extern  __shared__ float tile [];

     int tx=threadIdx.x;
     int ty=threadIdx.y;

    int col = blockIdx.x * OUTPUT_TILE_DIM  + threadIdx.x;
    int row = blockIdx.y *OUTPUT_TILE_DIM + threadIdx.y;
    int batch_index= threadIdx.z+blockIdx.z*blockDim.z;


    int shared_col=col-maskWidth/2;
    int shared_row=row-maskWidth/2;


    for(int channel=0;channel<depth;channel++)
   {
    if(shared_col<0||shared_col>=width||shared_row>=height||shared_row<0||batch_index>=batch_size)
       tile[ty*input_tile_dim+tx]=0;
       else
      tile[ty*input_tile_dim+tx]=static_cast<float>(input[batch_index*width*depth*height+shared_row*width*depth+shared_col*depth+channel]);

        __syncthreads();

 float sum = 0.0f;
if (tx < OUTPUT_TILE_DIM && ty < OUTPUT_TILE_DIM && col < width && row < height) {

    for (int i = 0; i < maskWidth; ++i) {
        for (int j =0; j < maskWidth; ++j) {

            int curr_row = i+ty;
            int curr_col =j+tx;
            if(curr_col<width&& curr_row<height&&curr_col>=0&&curr_row>=0)
            {

              sum+=mask[i*maskWidth+j]*static_cast<float>(tile[curr_row*input_tile_dim+curr_col]);

            }
        }
    }
    int outIdx = (batch_index*depth+channel)*height*width+row*width+col;
    output[outIdx] = sum;}
      __syncthreads();
}
}


float* read_mask(const char* file_path, int& maskWidth) {
    FILE* file = fopen(file_path, "r");
    if (!file) {
        fprintf(stderr, "Error: Could not open mask file %s\n", file_path);
        return nullptr;
    }

    // Read mask dimension (first line)
    if (fscanf(file, "%d", &maskWidth) != 1) {
        fprintf(stderr, "Error: Could not read mask dimension from %s\n", file_path);
        fclose(file);
        return nullptr;
    }

    float* mask = (float*)malloc(maskWidth * maskWidth * sizeof(float));
    if (!mask) {
        fprintf(stderr, "Error: Memory allocation failed for mask\n");
        fclose(file);
        return nullptr;
    }

    // Read mask values (subsequent lines)
    for (int i = 0; i < maskWidth; i++) {
        for (int j = 0; j < maskWidth; j++) {
            if (fscanf(file, "%f", &mask[i * maskWidth + j]) != 1) {
                fprintf(stderr, "Error: Invalid mask data at row %d, column %d\n", i+1, j+1);
                free(mask);
                fclose(file);
                return nullptr;
            }
        }
    }

    fclose(file);
    return mask;
}


uint8_t* load_images(const char* folder_path, int& width, int& height, int& channels, int batch_size,int & num_images, std::vector<std::string>& image_paths) {
    DIR *dir;
    struct dirent *ent;

    if ((dir = opendir(folder_path)) != NULL) {
        while ((ent = readdir(dir)) != NULL) {
            std::string filename = ent->d_name;
            if (filename.find(".jpg") != std::string::npos ||
                filename.find(".jpeg") != std::string::npos ||
                filename.find(".png") != std::string::npos) {
                image_paths.push_back(std::string(folder_path) + "/" + filename);
            }
        }
        closedir(dir);
    } else {
        perror("Could not open directory");
        return nullptr;
    }

    if (image_paths.empty()) {
        printf("No images found in %s\n", folder_path);
        return nullptr;
    }

   num_images=image_paths.size();
   uint8_t* h_input;
    // Load images into batch
    for (int i = 0; i < image_paths.size(); i++) {
        int img_width, img_height, img_channels;
        unsigned char* image_data = stbi_load(image_paths[i].c_str(), &img_width, &img_height, &img_channels, 0);

        if(i==0)
        {
           height=img_height;
        width=img_width;
        channels=img_channels;
           size_t input_size = image_paths.size() * height * width * channels * sizeof(uint8_t);
            h_input = (uint8_t*)malloc(input_size);

        }
        if (!image_data) {
            printf("Failed to load image: %s\n", image_paths[i].c_str());
            continue;
        }


        // Copy image data to batch (NHWC layout)
        for (int y = 0; y < height; y++) {
            for (int x = 0; x < width; x++) {
                for (int c = 0; c < channels; c++) {
                    int src_idx = (y * width + x) * channels + c;
                    int dst_idx = (i * height * width * channels) +
                                 (y * width * channels) +
                                 (x * channels) + c;
                    h_input[dst_idx] = image_data[src_idx];
                }
            }
        }

        stbi_image_free(image_data);
    }

    return h_input;
}

int main(int argc, char** argv)
{

   if (argc != 5) {
        printf("arguments are incorrect");
        return 1;
    }
    const char* input_folder = argv[1];
    const char* output_folder = argv[2];
    int batch_size = atoi(argv[3]);
    const char* mask_file = argv[4];



    int maskWidth;
    float*h_mask=read_mask(mask_file,maskWidth);
    if(!h_mask)
    {
      return 1;

    }


    int height,width,depth;
    uint8_t* h_input;
    int num_images;
    std::vector<std::string> input_paths;
    h_input=load_images(input_folder,width,height,depth,batch_size, num_images,input_paths);






    uint8_t* d_input;
    float* d_output;
    float* d_mask;

    //Allocate


    cudaMalloc(&d_mask, maskWidth * maskWidth * sizeof(float));
    cudaMemcpy(d_mask, h_mask, maskWidth * maskWidth * sizeof(float), cudaMemcpyHostToDevice);
      cudaMemcpyToSymbol(c_mask, h_mask, maskWidth * maskWidth * sizeof(float));

    for(int batch_start=0;batch_start<num_images;batch_start+=batch_size)
    {

        int current_batch_size = (batch_start + batch_size > num_images) ? num_images - batch_start : batch_size;
        size_t input_size = current_batch_size * height * width * sizeof(uint8_t)*depth;
        size_t output_size = current_batch_size * height * width * sizeof(float)*depth;;
         float* h_output = (float*)malloc(output_size);

            //copy to gpu
            cudaMalloc(&d_input, input_size);
            cudaMalloc(&d_output, output_size);


            cudaMemcpy(d_input,  &h_input[batch_start * width * height * depth], input_size, cudaMemcpyHostToDevice);

   dim3 block_size(OUTPUT_TILE_DIM+maskWidth-1, OUTPUT_TILE_DIM+maskWidth-1, 1);
   dim3 grid_size(
       (width + OUTPUT_TILE_DIM - 1) / OUTPUT_TILE_DIM,
       (height +OUTPUT_TILE_DIM - 1) / OUTPUT_TILE_DIM,
       current_batch_size

   );

              int sharedMemorySize = sizeof(float) * (OUTPUT_TILE_DIM + maskWidth - 1) * (OUTPUT_TILE_DIM + maskWidth - 1);

            conv3D_tiled<<<grid_size, block_size,sharedMemorySize>>>(d_input, width, height, depth, current_batch_size,
                d_output, d_mask, maskWidth);

            cudaError_t err = cudaGetLastError();
            if (err != cudaSuccess) {
            printf("CUDA error: %s\n", cudaGetErrorString(err));
            }

            cudaDeviceSynchronize();  // Required to flush printf output


            cudaMemcpy(h_output, d_output, output_size, cudaMemcpyDeviceToHost);


            save_images(output_folder,h_output,width,height,depth,current_batch_size,input_paths,batch_start);
            cudaFree(d_input);
            cudaFree(d_output);
            free(h_output);


    }



  free(h_mask);
    free(h_input);
    cudaFree(d_mask);

    return 0;


}


Overwriting kernel2.cu


In [106]:
!nvcc -arch=sm_75 kernel2.cu -o kernel2.out
!nvprof ./kernel2.out /content/input /content/kernel2 4 /content/mask.txt


==26577== NVPROF is profiling process 26577, command: ./kernel2.out /content/input /content/kernel2 4 /content/mask.txt
==26577== Profiling application: ./kernel2.out /content/input /content/kernel2 4 /content/mask.txt
==26577== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   89.01%  83.659ms         2  41.829ms  26.565ms  57.093ms  [CUDA memcpy DtoH]
                    6.51%  6.1222ms         4  1.5306ms     673ns  4.0872ms  [CUDA memcpy HtoD]
                    4.48%  4.2065ms         2  2.1032ms  1.4044ms  2.8021ms  conv3D_tiled(unsigned char const *, int, int, int, int, float*, float*, int)
      API calls:   64.57%  181.08ms         5  36.215ms  113.78us  180.44ms  cudaMalloc
                   32.99%  92.505ms         5  18.501ms  23.175us  58.107ms  cudaMemcpy
                    1.52%  4.2582ms         2  2.1291ms  1.4518ms  2.8064ms  cudaDeviceSynchronize
                    0.74%  2.0805ms         5  416

In [25]:
pip install torch torchvision opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [56]:
%%writefile kernel.py

import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import time
import sys

def load_mask(path):
    with open(path, "r") as f:
        lines = f.read().strip().split('\n')
        n = int(lines[0])
        mask_vals = [list(map(float, line.strip().split())) for line in lines[1:]]
        mask = torch.tensor(mask_vals, dtype=torch.float32)
    return mask.view(1, 1, 1, n, n), n  # Shape: [1, 1, 1, k, k]

def load_input_images(input_folder):
    input_files = sorted(os.listdir(input_folder))
    input_images = []
    for input_file in input_files:
        input_path = os.path.join(input_folder, input_file)
        input_images.append(Image.open(input_path).convert("RGB"))
    return input_images

def pytorch_convolution(input_images, kernel, kernel_size, batch_size=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using", device)

    transform = transforms.ToTensor()
    outputs = []
    timings = {'total_execution_time_ms': 0, 'batch_times_ms': []}

    kernel = kernel.to(device)

    for i in range(0, len(input_images), batch_size):
        batch_images = input_images[i:i+batch_size]
        batch_tensors = [transform(img).unsqueeze(0) for img in batch_images]
        batch_tensor = torch.cat(batch_tensors, dim=0).to(device)  # [B, 3, H, W]

        B, C, H, W = batch_tensor.shape

        # Reshape for 3D: Treat RGB as 'depth'
        batch_tensor = batch_tensor.view(B, C, 1, H, W)  # [B, C, D=1, H, W]
        batch_tensor = batch_tensor.permute(0, 2, 1, 3, 4)  # [B, 1, C, H, W]

        # Pad (H, W) only
        padding = kernel_size // 2
        batch_tensor_padded = F.pad(batch_tensor, (padding, padding, padding, padding), mode='constant', value=0)

        # Convolve
        batch_start_time = time.time()
        output = F.conv3d(batch_tensor_padded, kernel)
        batch_end_time = time.time()

        # Reshape back
        output = output.permute(0, 2, 3, 4, 1).squeeze(-1)  # [B, C, H, W]
        output = output.cpu().numpy()

        for img_array in output:
            outputs.append(img_array)

        timings['batch_times_ms'].append((batch_end_time - batch_start_time) * 1000)

    timings['total_execution_time_ms'] = sum(timings['batch_times_ms'])
    return outputs, timings

def save_output(output_images, output_path):
    os.makedirs(output_path, exist_ok=True)
    for i, img in enumerate(output_images):
        img = (img - img.min()) / (img.max() - img.min() + 1e-5)  # normalize to [0,1]
        img = (img * 255).astype(np.uint8)
        img = np.transpose(img, (1, 2, 0))  # [H, W, C]
        Image.fromarray(img).save(os.path.join(output_path, f"pytorch_output_{i}.jpg"))

def profile(timings):
    print("Total execution time of F.conv3d (ms):", timings['total_execution_time_ms'])
    for i, batch_time in enumerate(timings['batch_times_ms']):
        print(f"\tBatch {i+1} execution time (ms): {batch_time}")

def main():
    if len(sys.argv) != 5:
        print("Usage: python B_1_17_2_14.py <input_folder> <output_folder> <batch_size> <mask_file>")
        return

    input_folder = sys.argv[1]
    output_folder = sys.argv[2]
    batch_size = int(sys.argv[3])
    mask_file = sys.argv[4]

    kernel, kernel_size = load_mask(mask_file)
    input_images = load_input_images(input_folder)

    output_images, timings = pytorch_convolution(input_images, kernel, kernel_size, batch_size)
    save_output(output_images, output_folder)
    profile(timings)

if __name__ == "__main__":
    main()


Overwriting kernel.py


In [107]:
!python kernel.py /content/input /content/python 4 /content/mask.txt


Using cuda
Total execution time of F.conv3d (ms): 88.82594108581543
	Batch 1 execution time (ms): 86.7612361907959
	Batch 2 execution time (ms): 2.0647048950195312
